Ссылка на бота: https://t.me/TestYearProjectBot

In [1]:
# ссылки общедоступные
!gdown 1smER4yXJPN_KAQfua25kmDiNSnrtGF2F
!gdown 1KiYpxgmCb1w9e280qy6LxfzLg5d93W26

Downloading...
From: https://drive.google.com/uc?id=1smER4yXJPN_KAQfua25kmDiNSnrtGF2F
To: /content/eda_higgs.pdf
100% 1.07M/1.07M [00:00<00:00, 66.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KiYpxgmCb1w9e280qy6LxfzLg5d93W26
To: /content/EDA.md
100% 1.36k/1.36k [00:00<00:00, 4.25MB/s]


In [2]:
!pip install aiogram==3.4.0

In [3]:
token = '7753140188:AAFBJWmvssf1Cqc9K7vNSJRG7Ep8O7xyp-M'

In [4]:
import asyncio
#import logging
from aiogram import Bot, Dispatcher, F
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.filters import CommandStart
from aiogram.types import Message, BotCommand, InlineKeyboardMarkup, \
InlineKeyboardButton, CallbackQuery, FSInputFile

#logging.basicConfig(filename='logging.log',
#                    filemode='a',
#                    level=logging.DEBUG,
#                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#logger = logging.getLogger(__name__)
bot = Bot(token=token, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher(storage=MemoryStorage())

In [5]:
url = 'https://www.kaggle.com/competitions/higgs-boson/data https://www.kaggle.com/competitions/flavours-of-physics/data'

eda_buttons = [
    [InlineKeyboardButton(text='Higgs Boson', callback_data='higgs')],
    [InlineKeyboardButton(text='Finding τ → μμμ', callback_data='mumumu')]
    ]

higgs_buttons = [
    [InlineKeyboardButton(text='Получить выводы по EDA', callback_data='higgs_desc')],
    [InlineKeyboardButton(text='Получить PDF-файл', callback_data='higgs_pdf')],
    [InlineKeyboardButton(text='Перейти по ссылке на датасет', url=url)]
    ]

eda_markup = InlineKeyboardMarkup(inline_keyboard=eda_buttons, row_width=1)
higgs_markup = InlineKeyboardMarkup(inline_keyboard=higgs_buttons, row_width=1)

In [6]:
bot_commands = [
        BotCommand(command="start", description="Запустить бота"),
        BotCommand(command="help", description="Помощь"),
        BotCommand(command="eda", description="Просмотр EDA"),
    ]

In [7]:
import os

@dp.message(CommandStart())
async def cmd_start(message: Message):
    #logging.info(f'started chat {message.chat.id}')
    if os.path.exists('test.txt') and str(message.chat.id) in open('test.txt', 'r').read():
        await message.answer('Бот уже запущен!')
    else:
        open('test.txt', 'a').write(str(message.chat.id) + '\n')
        await message.answer('Доброго времени суток!')

@dp.message(F.text == '/eda')
async def cmd_eda(message: Message):
    #logging.info(f'activated /eda command, chat: {message.chat.id}')
    await message.answer('Выберите тип EDA', reply_markup=eda_markup)

@dp.message(F.text == '/help')
async def cmd_help(message: Message):
    #logging.info(f'activated /help command, chat: {message.chat.id}')
    await message.answer('\n'.join(['/' + bot_command.command + ' - ' + bot_command.description
                                   for bot_command in bot_commands]))

@dp.callback_query(F.data == 'higgs')
async def higgs_callback(callback: CallbackQuery):
    #logging.info(f'activated higgs callback, chat: {callback.message.chat.id}')
    await callback.message.edit_reply_markup()
    await callback.message.answer('Выберите нужное действие', reply_markup=higgs_markup)

@dp.callback_query(F.data == 'higgs_desc')
async def desc_callback(callback: CallbackQuery):
    #logging.info(f'activated higgs_desc callback, chat: {callback.message.chat.id}')
    await callback.message.edit_reply_markup()
    doc = open('EDA.md', 'r')
    await callback.message.answer(doc.read())

@dp.callback_query(F.data == 'higgs_pdf')
async def desc_callback(callback: CallbackQuery):
    #logging.info(f'activated higgs_pdf callback, chat: {callback.message.chat.id}')
    await callback.message.edit_reply_markup()
    await callback.message.answer_document(document=FSInputFile('eda_higgs.pdf'),
                                           caption='PDF-файл с EDA (Higgs Boson)')

@dp.callback_query(F.data == 'mumumu')
async def mumummu_callback(callback: CallbackQuery):
    #logging.info(f'activated mumumu callback, chat: {callback.message.chat.id}')
    await callback.message.edit_reply_markup()
    await callback.message.answer('Данный тип EDA недоступен!')

@dp.message()
async def cmd_not_found(message: Message):
    #logging.info(f'found unknown command, chat: {message.chat.id}')
    await message.answer('Введите /help, чтобы получить справку по командам')

#async def set_default_commands(bot):
#    await bot.set_my_commands(bot_commands)

In [ ]:
async def main():
    #dp.include_router(start_router)
    #await set_default_commands(bot)
    await bot.delete_webhook(drop_pending_updates=True)
    await dp.start_polling(bot)

await main()
#if __name__ == "__main__":
#    asyncio.run(main())